### 保存和恢复模型

保存 Tensorflow 的模型有许多方法——具体取决于您使用的 API。本指南使用 tf.keras， 一个高级 API 用于在 Tensorflow 中构建和训练模型。有关其他方法的实现，请参阅 TensorFlow 保存和恢复指南或保存到 eager。

### 安装并导入

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.0.0


### 获取示例数据集

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

# 手动platten 除以 255 编程一维向量
train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

### 定义模型  
简单顺序模型

In [3]:
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    return model

model = create_model()

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


### 在训练期间保存模型（以 checkpoints 形式保存）

### Checkpoint 回调用法
创建一个只在训练期间保存权重的 tf.keras.callbacks.ModelCheckpoint 回调：

In [5]:
checkpoint_path = 'trainning_1/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

# 创建权重保存回调
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    verbose=1
)

# 使用回调训练模型
model.fit(test_images, test_labels,epochs=10,
         validation_data=(test_images, test_labels),
         callbacks=[cp_callback])

# 这可能会生成与保存优化程序状态相关的警告。
# 这些警告（以及整个笔记本中的类似警告）是防止过时使用，可以忽略。

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 832/1000 [=======================>......] - ETA: 0s - loss: 1.6958 - accuracy: 0.4688
Epoch 00001: saving model to trainning_1/cp.ckpt
1000/1000 [==============================] - 0s 358us/sample - loss: 1.5523 - accuracy: 0.5150 - val_loss: 0.5168 - val_accuracy: 0.8560
Epoch 2/10
 800/1000 [=======================>......] - ETA: 0s - loss: 0.4412 - accuracy: 0.8662
Epoch 00002: saving model to trainning_1/cp.ckpt
1000/1000 [==============================] - 0s 359us/sample - loss: 0.4423 - accuracy: 0.8650 - val_loss: 0.2936 - val_accuracy: 0.9310
Epoch 3/10
 832/1000 [=======================>......] - ETA: 0s - loss: 0.2983 - accuracy: 0.9279
Epoch 00003: saving model to trainning_1/cp.ckpt
1000/1000 [==============================] - 0s 350us/sample - loss: 0.2886 - accuracy: 0.9270 - val_loss: 0.1917 - val_accuracy: 0.9620
Epoch 4/10
 768/1000 [======================>.......] - ETA: 0s - loss: 0.1977 - accuracy: 0.9544
Ep

In [8]:
!ls {checkpoint_dir}

'ls' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


In [9]:
# 创建新模型实例 测试评估
model1 = create_model()

# 评估
loss, acc = model1.evaluate(test_images, test_labels, verbose=2)
print('Untrained model1, accuracy: {:5.2f}%'.format(100 * acc))

1000/1 - 0s - loss: 2.2614 - accuracy: 0.0970
Untrained model1, accuracy:  9.70%


In [15]:
# 从checkpoint加载权重并重新评估
# 加载权重
model1.load_weights(checkpoint_path)

# 重新评估模型
loss,acc = model1.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:1.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.0264 - accuracy: 1.0000
Restored model, accuracy: 100.00%


### checkpoint回调选项-可调整回调评率，设定checkpoint时机

In [16]:
# 在文件名中包含 epoch (使用 `str.format`)
checkpoint_path = 'training_2/cp-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

# 创建一个回调，每 5 个 epochs 保存模型的权重
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5
)

# 创建一个新模型实例
model = create_model()

# 使用cp保存权重
model.save_weights(checkpoint_path.format(epoch=0))

# 使用新的回调 训练 模型
model.fit(
    train_images,
    train_labels,
    epochs=50,
    callbacks=[cp_callback],
    validation_data=(test_images, test_labels),
    verbose=0
)



Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [18]:
! ls {checkpoint_dir}

'ls' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


In [22]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2\\cp-0050.ckpt'

In [23]:
# 创建新模型
model = create_model()

# 加载latest checkpoint
model.load_weights(latest)

# 重新评估
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

1000/1 - 0s - loss: 0.4780 - accuracy: 0.8680
Restored model, accuracy: 86.80%


### 手动保存权重

使用 Model.save_weights 方法手动保存它们同样简单。默认情况下， tf.keras 和 save_weights 特别使用 TensorFlow checkpoints 格式 .ckpt 扩展名和 ( 保存在 HDF5 扩展名为 .h5 保存并序列化模型 )：

In [24]:
# 保存权重
model.save_weights('./checkpoints/my_checkpoint')

# 创建实例模型
model = create_model()

# 恢复权重
model.load_weights('./checkpoints/my_checkpoint')

# 评价模型
loss,acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.4780 - accuracy: 0.8680
Restored model, accuracy: 86.80%
